# Forward Kinematics

In [3]:
#Libraries that I am going to use
import numpy as np

In [2]:
#unit for l is [mm]
l0 = 275.5
l1 = 410
l2 = 207.3
l3 = 100 #this is at home pisition, it is usually a variable
l0+l1+l2+l3

992.8

In [11]:
def DenHartA(DH):
    '''
    This function makes matrix {j-1}^A_j from 
    Denavit-Hartenberg (DH) parameters

    DH should be as follows:
    [theta, d, a, alfa]

    theta = rotation around z axis
    d = moved in z direction
    a = moved in x direction
    alfa = rotation around x axis
    '''
    tta = DH[0]
    d = DH[1]
    a = DH[2]
    alfa = DH[3]
    A = np.array([
        [np.cos(tta), -np.sin(tta)*np.cos(alfa), np.sin(tta)*np.sin(alfa), a*np.cos(tta)],
        [np.sin(tta), np.cos(tta)*np.cos(alfa), -np.cos(tta)*np.sin(alfa), a*np.sin(tta)],
        [0, np.sin(alfa), np.cos(alfa), d],
        [0, 0, 0, 1]
    ])
    return A

def forward_kinematics(DH , n = None):
    """
    This function takes in the Denavit-Hartenberg parameter table
    and returns the position of the end effector with forward kinematics.
    Usually the links of the robot is the same as the lengt of DH. If not 
    you need to specify how many links with n = nr.links
    """
    if n is not None:
        links = n
    else :
        links = len(DH) #Find how many links there are, (how many DOF)

    A = np.zeros((links,4,4)) #Makes as many A matrix as there are links
    for i in range(links):
        A[i] = DenHartA(DH[i]) #Makes the A matrix of the DH parameters
    
    T = np.zeros(((links-1),4,4)) 
    for i in range(links-1):
        if i<1 :
            T[i] = A[i].dot(A[i+1])
        else :
            T[i] = T[i-1].dot(A[i+1])
            
    return T[-1].round(5) #T[-1] is when all the A matrixes are dotted together

def KINOVA(Tta_vec, n = None):
    """
    Here are the DH parameteres for this KINOVA robot arm.
    It takes a theta vector and completes the DH table and 
    return the position of the end effector usin forward kinematics
    """
    pihalf = np.pi/2
    alpha = pihalf
    DH = np.array([
        [Tta_vec[0]+pihalf, 0.2755, 0, alpha],
        [Tta_vec[1]+pihalf, 0, 0.410, 0],
        [Tta_vec[2]-pihalf, -0.0098, 0, -alpha],
        [Tta_vec[3]-pihalf, 0.3073, 0,0]
    ])
    
    if n is not None:
        return forward_kinematics(DH,n)
    else:
        return forward_kinematics(DH)

In [12]:
KINOVA(np.zeros(4))

array([[ 1.    ,  0.    ,  0.    , -0.0098],
       [ 0.    ,  1.    , -0.    ,  0.    ],
       [ 0.    ,  0.    ,  1.    ,  0.9928],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [5]:
q = [np.pi/2,0,np.pi/2,0]
KINOVA(q)

array([[ 0.    , -0.    ,  1.    ,  0.3073],
       [ 1.    , -0.    , -0.    , -0.0098],
       [ 0.    ,  1.    ,  0.    ,  0.6855],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [6]:
q = np.array([np.pi/2,3*np.pi/4,np.pi/4,np.pi])
KINOVA(q)

array([[-0.     , -1.     ,  0.     ,  0.28991],
       [-1.     ,  0.     ,  0.     , -0.0098 ],
       [-0.     , -0.     , -1.     , -0.32171],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [7]:
def DaVinci(Tta_vec, n = None):
    """
    This function completes the DH for Davinci surgical robot 
    with the the Theta vector and return the end position of 
    the end effector with forward kinematics
    """
    d = [0.45, 0.02, 0.4]
    a = [0.12, 0.30, 0.25]
    DH = np.array([
        [Tta_vec[0], 0, a[0], 0], #Link 1
        [Tta_vec[1]+np.pi/2, d[0], 0, np.pi/2], #Link 2
        [Tta_vec[2]+np.pi/2, 0, a[1], np.pi], #Link 3
        [Tta_vec[3]+np.pi, 0, a[2], 0], #Link 4
        [Tta_vec[4]+np.pi/2, d[1], 0, -np.pi/2], #Link 5
        [Tta_vec[5]+np.pi/2, d[2], 0, 0] #Link 6
    ])
    
    if n is not None:
        return forward_kinematics(DH,n)
    else:
        return forward_kinematics(DH)

In [8]:
q = np.zeros(6)
DaVinci(q)

array([[ 1. ,  0. , -0. ,  0.1],
       [-0. ,  1. , -0. , -0. ],
       [ 0. ,  0. ,  1. ,  0.9],
       [ 0. ,  0. ,  0. ,  1. ]])

In [9]:
q = np.array([np.pi/2, 0, 0, np.pi/4, np.pi/4, 0])
DaVinci(q)

array([[ 0.     , -0.     , -1.     , -0.22322],
       [ 1.     ,  0.     ,  0.     ,  0.1    ],
       [ 0.     , -1.     ,  0.     ,  0.57322],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [10]:
q = np.array([np.pi/2, np.pi/5, np.pi/4, 0, np.pi/4, 3*np.pi/4])
DaVinci(q)

array([[-0.15643,  0.98769,  0.     ,  0.04036],
       [-0.98769, -0.15643,  0.     ,  0.1246 ],
       [ 0.     , -0.     ,  1.     ,  0.88536],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [11]:
def forward_kinematics_daniel(Tta_vec):
    d = [0.45, 0.02, 0.4]
    a = [0.12, 0.30, 0.25]
    DH = np.array([
        [Tta_vec[0], 0, a[0], 0], #Link 1:
        [Tta_vec[1]+np.pi/2, d[0], 0, np.pi/2], #Link 2
        [Tta_vec[2]+np.pi/2, 0, a[1], np.pi], #Link 3
        [Tta_vec[3], 0, -a[2], 0], #Link 4
        [Tta_vec[4]+np.pi/2, d[1], 0, np.pi/2], #Link 5
        [Tta_vec[5]+np.pi/2, d[2], 0, 0] #Link 6
    ])
    
    A = np.zeros((6,4,4))
    for i in range(6):
        A[i] = DenHartA(DH[i])
   
    T = A[0].dot(A[1]).dot(A[2]).dot(A[3]).dot(A[4]).dot(A[5])
    
    return T.round(5)

In [12]:
q = [0,0,0,0,0,0]
forward_kinematics_daniel(q)

array([[-1. , -0. , -0. ,  0.1],
       [ 0. , -1. ,  0. ,  0. ],
       [-0. , -0. ,  1. ,  0.9],
       [ 0. ,  0. ,  0. ,  1. ]])

In [13]:
DaVinci(q)

array([[ 1. ,  0. , -0. ,  0.1],
       [-0. ,  1. , -0. , -0. ],
       [ 0. ,  0. ,  1. ,  0.9],
       [ 0. ,  0. ,  0. ,  1. ]])

In [14]:
q = np.array([np.pi/2, 0, 0, np.pi/4, np.pi/4, 0])
DaVinci(q)

array([[ 0.     , -0.     , -1.     , -0.22322],
       [ 1.     ,  0.     ,  0.     ,  0.1    ],
       [ 0.     , -1.     ,  0.     ,  0.57322],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [15]:
forward_kinematics_daniel(q)

array([[-0.     ,  0.     , -1.     , -0.22322],
       [-1.     , -0.     ,  0.     ,  0.1    ],
       [-0.     ,  1.     ,  0.     ,  0.57322],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [16]:
q = np.array([np.pi/2, np.pi/5, np.pi/4, 0, np.pi/4, 3*np.pi/4])
DaVinci(q)

array([[-0.15643,  0.98769,  0.     ,  0.04036],
       [-0.98769, -0.15643,  0.     ,  0.1246 ],
       [ 0.     , -0.     ,  1.     ,  0.88536],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [17]:
forward_kinematics_daniel(q)

array([[ 0.15643, -0.98769, -0.     ,  0.04036],
       [ 0.98769,  0.15643, -0.     ,  0.1246 ],
       [ 0.     ,  0.     ,  1.     ,  0.88536],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [18]:
def UR3(Tta_vec, n = None):
    """
    This function completes the DH for UR3 with the the Theta vector
    and return the end position of the end effector with forward kinematics
    """
    DH = np.array([
        [Tta_vec[0] + np.pi/2, 0.892, 0, np.pi/2],
        [Tta_vec[1] + np.pi/2, 0, 4.25, 0],
        [Tta_vec[2], 0, 3.92, np.pi],
        [Tta_vec[3] + np.pi/2, -1.093, 0, - np.pi/2],
        [Tta_vec[4], -0.9475, 0, np.pi/2],
        [Tta_vec[5], -0.825, 0, 0]
    ])
                
    if n is not None:
        return forward_kinematics(DH,n)
    else:
        return forward_kinematics(DH)

In [19]:
q = np.zeros(6)
UR3(q)

array([[ 0.    ,  0.    , -1.    ,  1.918 ],
       [ 1.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    , -1.    , -0.    , 10.0095],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [20]:
z = (89.2+425+392+94.75)/100
x = (109.3 + 82.5)/100
z, x

(10.009500000000001, 1.9180000000000001)

In [21]:
q = np.array([np.pi/2, 0, 0, np.pi/4, np.pi/4, 0])
UR3(q)

array([[-0.5    ,  0.70711, -0.5    , -0.25748],
       [ 0.70711, -0.     , -0.70711,  1.67636],
       [-0.5    , -0.70711, -0.5    , 10.14448],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])

In [22]:
q = np.array([np.pi/2, np.pi/5, np.pi/4, 0, np.pi/4, 3*np.pi/4])
UR3(q)

array([[-0.62018,  0.77662, -0.11062,  7.39692],
       [-0.5    , -0.5    , -0.70711,  1.67636],
       [-0.60446, -0.38323,  0.6984 ,  4.51559],
       [ 0.     ,  0.     ,  0.     ,  1.     ]])